# Step 1 - Mid-Market Option Valuation

This notebook familiarizes you with the basics of the `Strategy` class. A trading strategy receives order book updates ("snapshots") as well as trades from a futures contract and an corresponding at-the-money option. Snapshots can be used to update the valuation of either of the two instruments. Trades additionally can be used as a trigger to trade in either of the two instruments.

In this notebook, you build a simple custom strategy which handles option snapshots and trades and computes a mid-market valuation. Carefully read the description of the `Strategy` and related classes below before you get started.

## 1 Standard Imports

Don't change these. If you need custom imports than you need to put them directly into your strategy.

In [1]:
from dataclasses import dataclass
import datetime as dt
import numpy as np
from typing import Tuple
from trading_simulation.book_history import Snapshot, Trade
from trading_simulation.simulation import HARD_RISK_LIMITS, SOFT_RISK_LIMITS, PositionStats, TradingSimulation, SimulationSettings
from trading_simulation.strategy import Strategy, Valuation, Execution

## 2 Strategy Interface

### 2.1 Strategy

Trading strategies are implemented by creating a custom class which derives from `Strategy`.

```python
class Strategy:
    def handle_option_snapshot(self, snapshot: Snapshot) -> Valuation:
        return Valuation()

    def handle_option_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        return Valuation(), Execution()
```

At this moment, we only implement the two functions shown above which react to events in the option contract.

1. `handle_option_snapshot` is called whenever the state of the first-level orderbook for the option contract changes. It receives a `Snapshot` which represents the new order book state and returns a `Valuation` for both the futures and the option.
1. `handle_option_trade` is called when a trade in the option occurs. It receives a `Trade` which holds the traded price and volume and returns both an updated `Valuation` as well as an `Execution` which represents the action you want to take as a result of the `Trade`.

### 2.2 Snapshot

A `Snapshot` contains the following information about the state of the orderbook:

```python
class Snapshot:
    timestamp: int
    bid_price: float
    bid_volume: int
    ask_price: float
    ask_volume: float
    vola: float
    delta: float
    vega: float
```

In case of a futures contracts the last three fields are always equal to `vola = 0.0`, `delta = 1.0` and `vega = 0.0`. In case of the option, they contain the then-current mid-market volatility, delta and vega.

### 2.3 Trade

A `Trade` has to following structure:

```python
class Trade:
    timestamp: int
    traded_price: float
    traded_volume: int
```

The traded volume is always a strictly positive integer.

### 2.4 Valuation

A `Valuation` corresponds to a price for both the futures and the option contract.

```python
@dataclass(frozen=True)
class Valuation:
    futures_price: float = 0.0
    option_price: float = 0.0
```

### 2.5 Execution

The `Execution` contains the desired trade volume in the futures and the option.

```python
@dataclass(frozen=True)
class Execution:
    futures_volume: int = 0
    option_volume: int = 0
```

Both volumes need to be integers and a positive (negative) value indicates a buy (sell) trade. For the moment, we do not trade ourselves yet and always reply with `Execution()` which is the same as `Execution(0, 0)`.

## 3 Implementing an Options Valuation

We now implement a simple option valuation ourselves. The below code serves as a starting point. It is a valid implementation of the `Strategy` interface where the valuation of the option is always the last mid-market price.

In [2]:
@dataclass
class OptionValuation(Strategy):
    option_value: float = 10.0

    def handle_option_snapshot(self, snapshot: Snapshot) -> Valuation:
        self.option_value = self._calculate_mid(snapshot)
        return self.valuation

    def handle_option_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        return self.valuation, Execution()
    
    def _calculate_mid(self, snapshot: Snapshot) -> float:
        return 0.5 * (snapshot.bid_price + snapshot.ask_price)
    
    @property
    def valuation(self) -> Valuation:
        return Valuation(option_price=self.option_value)

## 4 Evaluating the Valuation

### 4.1 Running the Simulation

We can simulate our `OptionValuation` strategy on an in-sample dataset. Note that the final evaluation of your trading strategy will be run on a different dataset, so don't overfit! The `SimulationSettings` allow us to control how this simulation is run.

```python
class SimulationSettings:
    start_time: Optional[dt.time] = None
    end_time: Optional[dt.time] = None
    max_count: Optional[int] = None
    ignore_limits: bool = False
```

- The trading day normally starts at 9:00 AM and ends at 5:30 PM, but you can choose to simulate different time intervals by setting `start_time` and `end_time`.
- `max_count` controls how many market events we simulate at most. This is very useful to reduce the runtime of the simulation while you are still experimenting with your valuation. The full trading day has about two millon events and setting `max_count = 100000` is a good starting point.
- `ignore_limits` is useful when implementing the actual trading strategy. It allows you to ignore the delta and vega limits for the simulation. More on this later.

In [17]:
strategies = {
    'simple valuation': OptionValuation(),
}
simulation = TradingSimulation()
settings = SimulationSettings(max_count=100000)
result = simulation.simulate(strategies, settings)

### 4.2 Plotting the Valuation

Call `plot_valuations` on your result visualize your valuation strategy. It has a few optional arguments:
    
- `include_futures` and `include_option` control whether we include the futures and option data in the plot. For the moment we set this to `include_futures = False` and `include_option = True` as we don't have a futures valuation (yet).
- `count` sets the maximum number of points to plot. The data is being downsampled to this value. Setting `count = 1000` renders the plot reasonably quickly.
- `start_time` and `end_time` select the time slice to plot.

In [18]:
result.plot_valuations(include_futures=False, include_option=True, count=1000, start_time=dt.time(9, 0, 0), end_time=dt.time(17, 30, 0))

## 5 Next Steps

Think about how you could improve the simple valuation strategy. Consider e.g. how to incorporate the bid and offer volume in the valuation.